In [1]:
import os
from copy import deepcopy

import pandas as pd
from rectools.dataset import Dataset
from rectools.model_selection import TimeRangeSplitter
from rectools.models import PopularModel, RandomModel
from rectools.metrics import MAP, NDCG, Precision, Recall, MeanInvUserFreq, Serendipity

from dev_eval import calculate_metrics, read_kion_dataset, visualize

pd.set_option("display.max_columns", None)

# Data

Подгружаем датасет кион используя дополнительную функцию

In [2]:
kion_data = read_kion_dataset()
interactions = kion_data["interactions"]
users = kion_data["users"]
items = kion_data["items"]

# Параметры тестирования функций

Модели: `rectools.models.RandomModel(random_state=32)`, `rectools.models.PopularModel()` с параметрами по умолчанию

In [3]:
models = {
    "random": RandomModel(random_state=32),
    "popular": PopularModel(),
}

Метрики: 2 ранжирующие (MAP, NDCG), 2 классификационные (precision, recall), 2 beyond-accuracy (novelty, serendipity). Считаем по порогам 1, 5, 10. MAP обязательно

In [4]:
metrics = {
    "MAP@1": MAP(k=1),
    "MAP@5": MAP(k=5),
    "MAP@10": MAP(k=10),
    "NDCG@1": NDCG(k=1),
    "NDCG@5": NDCG(k=5),
    "NDCG@10": NDCG(k=10),
    "precision@1": Precision(k=1),
    "precision@5": Precision(k=5),
    "precision@10": Precision(k=10),
    "recall@1": Recall(k=1),
    "recall@5": Recall(k=5),
    "recall@10": Recall(k=10),
    "novelty@1": MeanInvUserFreq(k=1),
    "novelty@5": MeanInvUserFreq(k=5),
    "novelty@10": MeanInvUserFreq(k=10),
    "serendipity@1": Serendipity(k=1),
    "serendipity@5": Serendipity(k=5),
    "serendipity@10": Serendipity(k=10),
}

Сплиттер: `rectools.model_selection.TimeRangeSplitter`, 3 фолда для кросс-валидации по неделе, исключение холодных юзеров и айтемов и просмотренных айтемов

In [5]:
cv = TimeRangeSplitter(
    test_size="7D",  # по неделе
    n_splits=3,  # 3 фолда для кросс-валидации
    filter_already_seen=True,  # исключение просмотренных айтемов
    filter_cold_items=True,  # исключение холодных айтемов
    filter_cold_users=True,  #  исключение холодных юзеров
)

Визуализация рекомендаций и историй просмотров для юзеров `[666262, 672861, 955527]`. Для айтемов обязательно отражаем названия, жанры и количество просмотров в датасете (как для айтемов из истории взаимодействий каждого юзера, так и для айтемов из его рекомендаций)

In [6]:
users_list = [666262, 672861, 955527]
k_recos = 10

# Функция 1: метрики 

In [7]:
result_data = calculate_metrics(models, kion_data, metrics, cv, k_recos=k_recos, style=True)
result_data

  0%|          | 0/3 [00:00<?, ?it/s]

=======================================================|| Model: random | Fold: 0=======================================================
Fit time: 0.0 sec.
Recommend time: 5.88 sec.
Metrics time: 2.2 sec.
=======================================================|| Model: popular | Fold: 0=======================================================
Fit time: 1.25 sec.
Recommend time: 3.62 sec.
Metrics time: 1.89 sec.
=======================================================|| Model: random | Fold: 1=======================================================
Fit time: 0.0 sec.
Recommend time: 5.52 sec.
Metrics time: 2.02 sec.
=======================================================|| Model: popular | Fold: 1=======================================================
Fit time: 1.28 sec.
Recommend time: 3.81 sec.
Metrics time: 2.44 sec.
=======================================================|| Model: random | Fold: 2=======================================================
Fit time: 0.0 sec.
Recommend time: 6

# Функция 2: визуализация

In [8]:
item_data = ["title", "genres"]
k_recos = 10

In [9]:
for model_name in models:
    print(f"==={model_name}===")
    # extract model and train
    model = deepcopy(models[model_name])
    dataset_for_train = Dataset.construct(interactions.df)
    model.fit(dataset_for_train)
    visualize(model, kion_data, users_list, item_data, k_recos=k_recos, display=display)

===random===
Visual report
----------------------------------------------------------
User: 666262
Already watched films amount: 3
Display last 10 watched:



,item_id,datetime,weight,watched_pct,item_id_x,title,genres,item_id_y
2,12981,2021-05-14,10292.0,100.0,12981,Томирис,"боевики, драмы, историческое, военные",10370
0,7957,2021-05-12,2052.0,32.0,7957,Последний викинг,"боевики, историческое, приключения",746
1,4785,2021-05-12,1946.0,28.0,4785,Робин Гуд: Начало,"боевики, триллеры, приключения",485



Recommended films amount: 10
(Amount of all films: 15706)
Display first 10 recommendations:


,item_id,score,rank,item_id_x,title,genres,item_id_y
0,10101,10,1,10101,Возвращение Будулая,мелодрамы,99
1,619,9,2,619,Новые приключения Аладдина (жестовым языком),"зарубежные, комедии",1
2,12618,8,3,12618,Пропавшая грамота,"фэнтези, комедии",51
3,5967,7,4,5967,Братья вне игры,"драмы, спорт",262
4,4041,6,5,4041,Фрилансеры,"криминал, детективы, драмы, зарубежные, боевики",19
5,5701,5,6,5701,Алые паруса: Новая история,"комедии, мелодрамы",4
6,9738,4,7,9738,Женщина в беде 3,"детективы, мелодрамы",2
7,15247,3,8,15247,Гордость и предубеждение,"драмы, мелодрамы",150
8,10004,2,9,10004,Болванчики,"мультфильм, приключения, комедии",51
9,2816,1,10,2816,Избави нас от лукавого,"ужасы, триллеры, детективы",1370


----------------------------------------------------------
User: 672861
Already watched films amount: 2
Display last 10 watched:



...

,item_id,score,rank,item_id_x,title,genres,item_id_y
0,10440,202457.0,1,10440,Хрустальный,"триллеры, детективы",202457
1,15297,193123.0,2,15297,Клиника счастья,"драмы, мелодрамы",193123
2,9728,132865.0,3,9728,Гнев человеческий,"боевики, триллеры",132865
3,13865,122119.0,4,13865,Девятаев,"драмы, военные, приключения",122119
4,4151,91167.0,5,4151,Секреты семейной жизни,комедии,91167
5,3734,74803.0,6,3734,Прабабушка легкого поведения,комедии,74803
6,2657,68581.0,7,2657,Подслушано,"драмы, триллеры",68581
7,4880,55043.0,8,4880,Афера,комедии,55043
8,142,45367.0,9,142,Маша,"драмы, триллеры",45367
9,6809,40372.0,10,6809,Дуров,документальное,40372


----------------------------------------------------------
User: 955527
Already watched films amount: 4
Display last 10 watched:



,item_id,datetime,weight,watched_pct,item_id_x,title,genres,item_id_y
0,1183,2021-06-02,40.0,1.0,1183,Стань легендой! Бигфут Младший,"мультфильм, фэнтези, приключения, комедии",1587
2,4725,2021-06-02,255.0,4.0,4725,Лобановский навсегда,"спорт, биография, документальное",683
3,1238,2021-06-02,556.0,7.0,1238,Диего Марадона,"спорт, биография, документальное",691
1,13371,2021-05-04,686.0,11.0,13371,Пеле: Рождение легенды,"драмы, спорт, биография",945



Recommended films amount: 10
(Amount of all films: 15706)
Display first 10 recommendations:


,item_id,score,rank,item_id_x,title,genres,item_id_y
0,10440,202457.0,1,10440,Хрустальный,"триллеры, детективы",202457
1,15297,193123.0,2,15297,Клиника счастья,"драмы, мелодрамы",193123
2,9728,132865.0,3,9728,Гнев человеческий,"боевики, триллеры",132865
3,13865,122119.0,4,13865,Девятаев,"драмы, военные, приключения",122119
4,4151,91167.0,5,4151,Секреты семейной жизни,комедии,91167
5,3734,74803.0,6,3734,Прабабушка легкого поведения,комедии,74803
6,2657,68581.0,7,2657,Подслушано,"драмы, триллеры",68581
7,4880,55043.0,8,4880,Афера,комедии,55043
8,142,45367.0,9,142,Маша,"драмы, триллеры",45367
9,6809,40372.0,10,6809,Дуров,документальное,40372


Проверка, если попросить выводить только первые пять (меняем `k_display`, не `k_recos`)

In [11]:
for model_name in models:
    print(f"==={model_name}===")
    # extract model and train
    model = deepcopy(models[model_name])
    dataset_for_train = Dataset.construct(interactions.df)
    model.fit(dataset_for_train)
    visualize(model, kion_data, users_list, item_data, k_recos=k_recos, display=display, k_display=5)

===random===
Visual report
----------------------------------------------------------
User: 666262
Already watched films amount: 3
Display last 5 watched:



,item_id,datetime,weight,watched_pct,item_id_x,title,genres,item_id_y
2,12981,2021-05-14,10292.0,100.0,12981,Томирис,"боевики, драмы, историческое, военные",10370
0,7957,2021-05-12,2052.0,32.0,7957,Последний викинг,"боевики, историческое, приключения",746
1,4785,2021-05-12,1946.0,28.0,4785,Робин Гуд: Начало,"боевики, триллеры, приключения",485



Recommended films amount: 10
(Amount of all films: 15706)
Display first 5 recommendations:


,item_id,score,rank,item_id_x,title,genres,item_id_y
0,10101,10,1,10101,Возвращение Будулая,мелодрамы,99
1,619,9,2,619,Новые приключения Аладдина (жестовым языком),"зарубежные, комедии",1
2,12618,8,3,12618,Пропавшая грамота,"фэнтези, комедии",51
3,5967,7,4,5967,Братья вне игры,"драмы, спорт",262
4,4041,6,5,4041,Фрилансеры,"криминал, детективы, драмы, зарубежные, боевики",19


----------------------------------------------------------
User: 672861
Already watched films amount: 2
Display last 5 watched:



...

,item_id,score,rank,item_id_x,title,genres,item_id_y
0,10440,202457.0,1,10440,Хрустальный,"триллеры, детективы",202457
1,15297,193123.0,2,15297,Клиника счастья,"драмы, мелодрамы",193123
2,9728,132865.0,3,9728,Гнев человеческий,"боевики, триллеры",132865
3,13865,122119.0,4,13865,Девятаев,"драмы, военные, приключения",122119
4,4151,91167.0,5,4151,Секреты семейной жизни,комедии,91167


----------------------------------------------------------
User: 955527
Already watched films amount: 4
Display last 5 watched:



,item_id,datetime,weight,watched_pct,item_id_x,title,genres,item_id_y
0,1183,2021-06-02,40.0,1.0,1183,Стань легендой! Бигфут Младший,"мультфильм, фэнтези, приключения, комедии",1587
2,4725,2021-06-02,255.0,4.0,4725,Лобановский навсегда,"спорт, биография, документальное",683
3,1238,2021-06-02,556.0,7.0,1238,Диего Марадона,"спорт, биография, документальное",691
1,13371,2021-05-04,686.0,11.0,13371,Пеле: Рождение легенды,"драмы, спорт, биография",945



Recommended films amount: 10
(Amount of all films: 15706)
Display first 5 recommendations:


,item_id,score,rank,item_id_x,title,genres,item_id_y
0,10440,202457.0,1,10440,Хрустальный,"триллеры, детективы",202457
1,15297,193123.0,2,15297,Клиника счастья,"драмы, мелодрамы",193123
2,9728,132865.0,3,9728,Гнев человеческий,"боевики, триллеры",132865
3,13865,122119.0,4,13865,Девятаев,"драмы, военные, приключения",122119
4,4151,91167.0,5,4151,Секреты семейной жизни,комедии,91167


Работает!